In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
# from common.models.place import Event
from langchain_core.language_models.chat_models import BaseChatModel
from loguru import logger
import json
from pydantic import BaseModel
from langchain_ollama import ChatOllama
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field, ConfigDict

In [2]:
load_dotenv()

model = ChatOllama(
	base_url=os.environ.get("OLLAMA_HOST"),
	model="llama3.1:8b",
	num_gpu=-1,
	validate_model_on_init=True,
	temperature=0.3
)

In [3]:
class EventInstanceName(BaseModel):
    """
    Model output for generating a generic snake_case narrative event identifier.
    """

    model_config = ConfigDict(
        str_strip_whitespace=True,
        extra="forbid"
    )

    instance_name: str = Field(
        ...,
        description=(
            "Generic narrative event identifier in snake_case. "
            "Must correspond to the event type and contain no story-specific details."
        ),
        examples=[
            "antagonist_rushes_to_finish_line",
            "hero_passing_antagonist",
            "loss_of_safe_space",
            "magical_helper_grants_wish",
            "hero_works_hard"
        ],
        min_length=1
    )


In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template(
    template="""You are an expert narrative annotation assistant.
Your task is to generate a GENERIC instance name for a narrative event.

You will be given:
- The event type
- The event text
- A brief reasoning explaining why this is that event type

Instructions:
- Output ONLY a generic instance name.
- Do NOT include specific characters, places, objects, or story-specific details.
- The name must be abstract and reusable.
- Use snake_case.
- Do NOT repeat or summarize the input fields.
- Do NOT output explanations or additional fields.

Output format:
{{
  "instance_name": "generic_event_name"
}}
"""
)


In [11]:
human_prompt = HumanMessagePromptTemplate.from_template(
    """Event type:
{event_type}

Event text:
\"\"\"{event_text}\"\"\"

Reasoning:
{thinking}
"""
)


In [12]:
event_instance_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    human_prompt,
])


In [13]:
def extract_event_instance_Name(model: BaseChatModel, event_type: str, event_text:str, thinking:str = "" ):
	event_chain = event_instance_prompt | model.with_structured_output(EventInstanceName)
	response = event_chain.invoke({
		"event_type": event_type,
		"event_text": event_text,
		"thinking": thinking
	})
	return response.instance_name

In [ ]:
story_events = [
    "Three little pigs decide to leave their mother's house and build their own homes to live independently.",
    "The first pig quickly builds a house made of straw and relaxes, confident it will be enough.",
    "The second pig builds a house of wood, stronger than straw but still hastily constructed.",
    "The third pig carefully builds a solid house made of bricks, taking time and effort.",
    "A big bad wolf arrives and blows down the straw house, forcing the first pig to flee.",
    "The wolf attacks the wooden house and blows it down, chasing the first two pigs away.",
    "Both pigs run to the brick house, where the wolf fails to blow it down.",
    "The wolf attempts to enter through the chimney but falls into a pot of boiling water.",
    "The wolf is defeated, and the three pigs live safely together, having learned the value of hard work and preparation."
]

event_types = [
    "move.setup.initial_situation",
    "move.setup.initial_situation",
    "move.setup.initial_situation",
    "move.setup.initial_situation",
    "move.conflict.hero_interdiction.villainy",
    "move.conflict.struggle.fight",
    "move.conflict.struggle.fight",
    "resolution.victory.villain_defeated",
    "resolution.transfiguration.psychological_transformation"
]


thinking = """The text describes the setup of the story, where the main characters (three little pigs) take an action (deciding to leave their mother's house and build their own homes), which drives the narrative.
The given text describes a setup for the story, where the main characters (three little pigs) take an action (leaving their mother's house), which drives the narrative forward. This is more specific than 'resolution' as it doesn't involve conflict resolution or closure.
The narrative event described is the setup or preparation for the story, where the main characters (three little pigs) take an action (leaving home and building their own houses), which will likely lead to conflict.
The text describes a setup or preparation for the main action, as it introduces the situation and characters before any conflict arises.
The given text sets up the situation for a story, introducing the main characters (three little pigs) and their decision to act (leave their mother's house and build their own homes). This is the initial setup or preparation for the conflict that will likely arise.
The narrative event described is the setup or preparation for the story, where the main characters (three little pigs) take an action (leaving home and building their own houses), which will likely lead to conflict.
The given text sets up the situation for a story, introducing the main characters (three little pigs) and their decision to act (leave their mother's house and build their own homes). This is the initial setup or preparation for the conflict that will likely arise.
The narrative event described is the setup or preparation for the story, where the main characters (three little pigs) take an action (leaving home and building their own houses), which will likely lead to conflict.
The narrative event described is the setup or preparation for the story, where the main characters (three little pigs) take an action (leaving home and building their own houses), which will likely lead to conflict.
The narrative event described is the setup or preparation for the story, where the main characters (three little pigs) take an action (leaving home and building their own houses), which will likely lead to conflict.
The given text sets up the situation for a story, introducing the main characters (three little pigs) and their decision to act (leave their mother's house and build their own homes). This is the initial setup or preparation for the conflict that will likely arise.
The given text sets up the situation for a story, introducing the main characters (three little pigs) and their decision to act (leave their mother's house and build their own homes). This is the initial setup or preparation for the conflict that will likely arise.
The given text sets up the situation for a story, introducing the main characters (three little pigs) and their decision to act (leave their mother's house and build their own homes). This is the initial setup or preparation for the conflict that will likely arise.
The given text sets up the situation for a story, introducing the main characters (three little pigs) and their decision to act (leave their mother's house and build their own homes). This is the initial setup or preparation for the conflict that will likely arise.
The given text sets up the situation for a story, introducing the main characters (three little pigs) and their decision to act (leave their mother's house and build their own homes). This is the initial setup or preparation for the conflict that will likely arise."""

In [16]:
t2= """
The text describes the closure of the story with conflict resolution, hero's victory, recognition, punishment of the villain (the wolf), transformations, and symbolic closure such as the pigs living safely together.
The narrative has reached its conclusion with conflict resolution (the wolf's defeat) and a transformation (the pigs' newfound appreciation for hard work and preparation).
The text describes a conclusion to the conflict between the wolf and the three pigs. The outcome is that the wolf is defeated and the pigs are safe.
The narrative has reached its conclusion with conflict resolution, as the wolf is defeated and the pigs are safe. This indicates a specific type of closure.
The text describes a closure of the story with conflict resolution (the wolf being defeated) and transformation (the pigs learning from their experience).
The narrative has reached its conclusion with conflict resolution (the wolf's defeat) and a transformation (the pigs' newfound appreciation for hard work and preparation).
The narrative has reached its conclusion with conflict resolution (the wolf's defeat) and a transformation (the pigs' newfound appreciation for hard work and preparation).
The narrative has reached its conclusion with conflict resolution (the wolf's defeat) and a transformation (the pigs' newfound appreciation for hard work and preparation).
The narrative has reached its conclusion with conflict resolution (the wolf's defeat) and a transformation (the pigs' newfound appreciation for hard work and preparation).
The narrative has reached its conclusion with conflict resolution (the wolf's defeat) and a transformation (the pigs' newfound appreciation for hard work and preparation).
The narrative has reached its conclusion with conflict resolution (the wolf's defeat) and a transformation (the pigs' newfound appreciation for hard work and preparation).
The narrative has reached its conclusion with conflict resolution (the wolf's defeat) and a transformation (the pigs' newfound appreciation for hard work and preparation).
The narrative has reached its conclusion with conflict resolution (the wolf's defeat) and a transformation (the pigs' newfound appreciation for hard work and preparation).
The narrative has reached its conclusion with conflict resolution (the wolf's defeat) and a transformation (the pigs' newfound appreciation for hard work and preparation).
The narrative has reached its conclusion with conflict resolution (the wolf's defeat) and a transformation (the pigs' newfound appreciation for hard work and preparation).
"""

In [17]:

extract_event_instance_Name(model,"psychological_transformation",story_events[-1],t2)

'closure_of_conflict'